In [1]:
# 앙상블 학습
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
train_input, test_input, train_target,test_target=train_test_split(data,target,test_size=0.2, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)
# return_train_score은 검증점수와 훈련세트에 대한 점수도 반환
scores=cross_validate(rf,train_input,train_target,return_train_score=True, n_jobs=-1) # 교차검증 수행
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 훈련세트, 테스트 세트에 대한 점수

0.9973541965122431 0.8905151032797809


In [3]:
rf.fit(train_input, train_target) # 랜덤 포레스트 훈련
print(rf.feature_importances_) # 특성 중요도 계산 alcohol, sugar, pH

[0.23167441 0.50039841 0.26792718]


In [4]:
rf=RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # oob : 서브샘플 추출후 남은 데이터들
rf.fit(train_input, train_target) # oob 평가 훈련
print(rf.oob_score_)

0.8934000384837406


In [5]:
# 엑스트라 트리
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores=cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_) # 특성 중요도

[0.20183568 0.52242907 0.27573525]


In [7]:
# 그레이디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb=GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


In [10]:
#히스토그램 기반 그레이디언트 부스팅 256개 정수구간으로 나눔
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result=permutation_importance(hgb, train_input, train_target,
                             n_repeats=10, random_state=42, n_jobs=-1)# permutation_importance는 특성 중요도,평균,표준편차를 담고있다
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result=permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

In [14]:
from xgboost import XGBClassifier # XGBoost 사용
xgb=XGBClassifier(tree_method='hist', random_state=42)
scores=cross_validate(xgb,train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[17:57:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\drepi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:57:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:57:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [15]:
from lightgbm import LGMClassifier # lightgbm 사용
lgb=LGMClassifier(random_state=42)
scores=cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

ImportError: cannot import name 'LGMClassifier' from 'lightgbm' (C:\Users\drepi\Anaconda3\lib\site-packages\lightgbm\__init__.py)

In [16]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.
